In [12]:
import pickle
import os
import numpy as np
from tqdm import tqdm
from scipy.io import wavfile
from python_speech_features import mfcc
from keras.models import load_model
import pandas as pd
from sklearn.metrics import accuracy_score
from cfg import Config

In [20]:
def predictions(audio_dir):
    y_true=[]
    y_pred=[]
    fn_prob={}
    print('Extracting features from audio')
    for i in tqdm(os.listdir(audio_dir)):
        rate, wav= wavfile.read(os.path.join(audio_dir,i))
        label=fn2class[i]
        c=classes.index(label)
        y_prob = []
        
        for i in range(0, wav.shape[0]-config.step, config.step):
            sample= wav[i:i+config.step]
            x=mfcc(sample, rate, numcep=config.nfeat,nfilt=config.nfilt, nfft=config.nfft)
            x=(x - config.min) /(config.max - config.min)
            
            if config.mode=='conv':
                x=x.reshape(1,x.shape[1], x.shape[0],1)
            
            elif config.mode=='time':
                x=np.expand_dims(x,axis=0)
            
            y_hat= model.predict(x)
            y_prob.append(y_hat)
            y_pred.append(np.argmax(y_hat))
            y_true.append(c)
        
        fn_prob[i] = np.mean(y_prob, axis=0).flatten()
        
    return y_true, y_pred, fn_prob

In [18]:
config=Config(mode='conv')

df=pd.read_csv('instruments.csv')
classes=list(np.unique(df.label))
fn2class=dict(zip(df.fname,df.label))
p_path=os.path.join('pickles','conv.p')



model=load_model(config.model_path)

with open(p_path, 'rb') as handle:

    config=pickle.load(handle)


y_true, y_pred, fn_prob = predictions('clean')

acc_score= accuracy_score(y_true=y_true, y_pred= y_pred)

y_probs=[]

for i , row in df.iterrows():
    y_prob= fn_prob[row.fname]
    y_probs.append(y_prob)
    for c, p in zip(classes, y_prob):
        df.at[i, c]= p
y_pred = [classes[np.argmax(y)] for y in y_probs]
df['y_pred']=y_pred
df.to_csv("predictions.csv", index=False)

Extracting features from audio


 30%|████████████████████████                                                         | 89/300 [00:07<00:16, 12.47it/s]C:\Users\pavig\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\pavig\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [00:28<00:00, 10.69it/s]


In [19]:
config=Config(mode='time')

df=pd.read_csv('instruments.csv')
classes=list(np.unique(df.label))
fn2class=dict(zip(df.fname,df.label))
p_path=os.path.join('pickles','time.p')


model=load_model(config.model_path)

with open(p_path, 'rb') as handle:
    config=pickle.load(handle)

y_true, y_pred, fn_prob = predictions('clean')
acc_score= accuracy_score(y_true=y_true, y_pred= y_pred)

y_probs=[]

for i , row in df.iterrows():
    y_prob= fn_prob[row.fname]
    y_probs.append(y_prob)
    for c, p in zip(classes, y_prob):
        df.at[i, c]= p
y_pred = [classes[np.argmax(y)] for y in y_probs]
df['y_pred']=y_pred
df.to_csv("predictions_rnn.csv", index=False)

Extracting features from audio


 30%|████████████████████████▎                                                        | 90/300 [00:18<00:34,  6.03it/s]C:\Users\pavig\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\pavig\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:09<00:00,  4.32it/s]
